## Extract Image&Label Patches For Training and Test

This notebook is to extract Image&Label patches from original images and labels.

The only thing you need to do, is to set up the `img_dir`, `ann_dir` and `out_dir`.

In [4]:
import glob
import os
from shutil import copyfile

import scipy.io as sio

import cv2
import numpy as np
import itertools
import matplotlib.pyplot as plt
from tqdm import tqdm

from misc.patch_extractor import PatchExtractor
from misc.utils import rm_n_mkdir

from config import Config

In [11]:
cfg = Config()

img_ext = '.png'
label_ext = '.png'

extract_type = 'mirror' # 'valid' for fcn8 segnet etc.
                        # 'mirror' for u-net etc.
# check the patch_extractor.py 'main' to see the different

# orignal size (win size) - input size - output size (step size)
# 512x512 - 256x256 - 256x256 fcn8, dcan, segnet
# 536x536 - 268x268 - 84x84   unet, dist
# 540x540 - 270x270 - 80x80   xy, hover
# 504x504 - 252x252 - 252x252 micronetcd tr
step_size = [256, 256] # should match self.train_mask_shape (config.py) 
win_size  = [512, 512] # should be at least twice time larger than 
                       # self.train_base_shape (config.py) to reduce 
                       # the padding effect during augmentation

xtractor = PatchExtractor(win_size, step_size)

### Paths to data - these need to be modified according to where the original data is stored
img_ext = '.png'
img_dir = '/home1/gzy/BreastCancerSeg/DATA/Test/images/'
ann_dir = '/home1/gzy/BreastCancerSeg/DATA/Test/masks/'
####
out_dir = "/home1/gzy/BreastCancerSeg/DATA/Test/%dx%d_%dx%d" % \
                    (win_size[0], win_size[1], step_size[0], step_size[1])

file_list = glob.glob('%s/*%s' % (img_dir, img_ext))
file_list.sort()

rm_n_mkdir(out_dir)

['/home1/gzy/BreastCancerSeg/DATA/Train/512x512_256x256/']
['/home1/gzy/BreastCancerSeg/DATA/Test/512x512_256x256/']


In [12]:
for filename in tqdm(file_list):
    filename = os.path.basename(filename)
    basename = '.'.join(filename.split('.')[:-1])

    img = cv2.imread(img_dir + basename + img_ext)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    label = cv2.imread(ann_dir + basename + label_ext, 0)
    label = label - 1
    # resize img and label to 1/4
    h, w ,c = img.shape
    img = cv2.resize(img, (int(w/4), int(h/4)), interpolation=cv2.INTER_CUBIC)
    label = cv2.resize(label, (int(w/4), int(h/4)), interpolation=cv2.INTER_NEAREST)
    
    ann = np.expand_dims(label, axis=-1)
    ann = ann.astype('int32')

    img = np.concatenate([img, ann], axis=-1)
    sub_patches = xtractor.extract(img, extract_type)
    for idx, patch in enumerate(sub_patches):
        np.save("{0}/{1}_{2:03d}.npy".format(out_dir, basename, idx), patch)

100%|██████████| 44/44 [01:51<00:00,  2.54s/it]
